In [ ]:
#모듈
import math
import pandas as pd
import numpy as np

try:
    import cufflinks as cf
except:
    !pip install cufflinks
    import cufflinks as cf

cf.go_offline()

from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 데이터 다루기
pd.set_option("Display.max_columns",200)
import re
import time

# 데이터 시각화
import seaborn as sns

# 사용할 머신러닝 모델들 - RandomForest, AdaBoost, GradientBoost, ExtraTrees
import sklearn
from sklearn.metrics import log_loss
from sklearn.svm import SVC

In [ ]:
#명목형 변수 더미화
def dummify_category_cols(df,cols):
    dummies = []
    new_df = df.copy()

    for idx in range(len(cols)):
        new_df = new_df.join(pd.get_dummies(total[cols[idx]], prefix = cols[idx]))
        new_df = new_df.drop(cols[idx], axis =1)
    return new_df

In [ ]:
#숫자형 변수 범주화
def dummify_number_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: int(x)//num * num)
    return new_df


In [ ]:
#명목형 변수의 연속형 변환 - 변수의 차원을 낮추기 위하여
def num_category(df, col, chg) :
    new_df = df.copy()
    set = {}
    
    for idx in range(len(chg)) :
        set[chg[idx]] = idx
        
    new_df[col] = new_df[col].map(set)
    
    return new_df        

In [ ]:
#숫자형 변수 일정 이상은 묶어버리기
def number_group_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: num if x> num else x )
    return new_df


In [ ]:
#RMS 구하기
def RMSE(y_true,y_pred):
    
    squaredErrors = (y_pred - y_true)**2
    MSE = squaredErrors.mean()
    
    return math.sqrt(MSE)

In [ ]:
#본격적으로 시작

In [ ]:
training = pd.read_csv("house_price.csv")
test = pd.read_csv("test.csv")

#training = training[np.isfinite(training["GarageYrBlt"])]

In [ ]:
total = pd.concat([training, test]).reset_index()
del total["index"]

In [ ]:
total.head(2)

In [ ]:
total.columns

In [ ]:
#선택된 변수
category_selected = ["Neighborhood", "OverallQual", 
            "MasVnrType", "ExterQual", "ExterCond",
            "FullBath", 
            "TotRmsAbvGrd", "Fireplaces",
            "GarageCars"]

extra_selected = [ "GarageCars", "OverallQual","OverallCond"]

number_selected = ["GarageYrBlt","YearBuilt", "YearRemodAdd", "BsmtFinSF1", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "GrLivArea", 'GarageArea', "LotArea", "MasVnrArea", "WoodDeckSF", "OpenPorchSF"]

category_selected = [x for x in category_selected if x not in extra_selected]

total_selected = category_selected + number_selected + extra_selected

selected_with_SalePrice = total_selected +["SalePrice"]

In [ ]:
new_total = total[selected_with_SalePrice].copy()
new_total.head(2)

In [ ]:
#임시 전처리

In [ ]:
#전처리1 - 연속형데이터 중 null값이 있는 행 모두 버려주기
new_total[number_selected].isnull().sum()

In [ ]:
new_total["YearBuilt"] = new_total["YearBuilt"].fillna(new_total.YearBuilt.mean())

In [ ]:
new_total["BsmtFinSF1"] = new_total["BsmtFinSF1"].fillna(new_total.BsmtFinSF1.mean())

In [ ]:
new_total["BsmtUnfSF"] = new_total["BsmtUnfSF"].fillna(new_total.BsmtUnfSF.mean())

In [ ]:
new_total["TotalBsmtSF"] = new_total["TotalBsmtSF"].fillna(new_total.TotalBsmtSF.mean())

In [ ]:
new_total["GarageArea"] = new_total["GarageArea"].fillna(new_total.GarageArea.mean())

In [ ]:
new_total["GarageYrBlt"] = new_total["GarageYrBlt"].fillna(new_total.GarageYrBlt.mean())

In [ ]:
#new_total["Electrical"] = new_total["Electrical"].fillna("SBrkr")

In [ ]:
new_total = new_total.fillna("404")

In [ ]:
new_total.isnull().sum()

In [ ]:
new_total.head()

In [ ]:
#명목형 변수를 연속형으로 바꿔주는 것

In [ ]:
#new_total = num_category(new_total, "Electrical", ["Mix", "FuseP", "FuseF", "FuseA", "SBrkr"])

In [ ]:
#new_total = num_category(new_total, "Functional", ["404", "Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"])
#new_total.Functional.unique()

In [ ]:
#변수끼리 합쳐주는 것

In [ ]:
#임시 전처리

In [ ]:
new_total = new_total.assign(Is_Remod =  lambda x : x["YearBuilt"] != x["YearRemodAdd"])
new_total["Is_Remod"] = new_total.YearRemodAdd.map(lambda x : 1 if x == True else 0)
new_total["after"] = 2011 - new_total["YearRemodAdd"]
new_total = new_total.drop("YearRemodAdd", axis = 1)

In [ ]:
#new_total = new_total.assign(Is_bsmt_unfinish = lambda x : x["BsmtFinType1"] == 0)
#new_total["Is_bsmt_unfinish"] = new_total.Is_bsmt_unfinish.map(lambda x : 1 if x == True else 0)

In [ ]:
#new_total = new_total.assign(Is_Fireplaces = lambda x : x["Fireplaces"] != 0)
#new_total["Is_Fireplaces"] = new_total.Is_Fireplaces.map(lambda x : 1 if x == True else 0)

In [ ]:
#new_total = new_total.assign(Is_GarageFinish = lambda x : x["GarageFinish"] != "Unf")
#new_total["Is_GarageFinish "] = new_total.Is_GarageFinish .map(lambda x : 1 if x == True else 0)

In [ ]:
new_total["Total_Area"] = new_total["GrLivArea"] + new_total["LotArea"] + new_total["GarageArea"] + new_total["TotalBsmtSF"]

In [ ]:
new_total["GrLivArea"] = new_total["GrLivArea"].map(lambda x : math.log(x))

In [ ]:
new_total["LotArea"] = new_total["LotArea"].map(lambda x : math.log(x))

In [ ]:
new_total["GarageArea"] = new_total["GarageArea"].map(lambda x : math.log(x+1))

In [ ]:
new_total["TotalBsmtSF"] = new_total["TotalBsmtSF"].map(lambda x : math.log(x+1))

In [ ]:
new_total["Total_Area"] = new_total["Total_Area"].map(lambda x : math.log(x+1))

In [ ]:
new_total["BsmtFinSF1"] = new_total["BsmtFinSF1"].map(lambda x : math.log(x+1))

In [ ]:
new_total["BsmtUnfSF"] = new_total["BsmtUnfSF"].map(lambda x : math.log(x+1))

In [ ]:
new_total["1stFlrSF"] = new_total["1stFlrSF"].map(lambda x : math.log(x+1))

In [ ]:
new_total["2ndFlrSF"] = new_total["2ndFlrSF"].map(lambda x : math.log(x+1))

In [ ]:
new_total["is_masvnr"] = new_total.MasVnrArea.map(lambda x : 0 if int(x) == 0 else 1)
new_total["is_wood"] = new_total.WoodDeckSF.map(lambda x : 0 if int(x) == 0 else 1)
new_total["is_openporch"] = new_total.OpenPorchSF.map(lambda x : 0 if int(x) == 0 else 1)

In [ ]:
new_total["MasVnrArea"] = new_total["MasVnrArea"].map(lambda x : math.log(int(x)+1))
new_total["WoodDeckSF"] = new_total["WoodDeckSF"].map(lambda x : math.log(int(x)+1))
new_total["OpenPorchSF"] = new_total["OpenPorchSF"].map(lambda x : math.log(int(x)+1))

In [ ]:
new_total["SalePrice"] = new_total.SalePrice.map(lambda x: math.log(int(x)+1))

In [ ]:
#확인

In [ ]:
new_total.head(2)

In [ ]:
#히트맵으로 확인
#training = training = new_total.iloc[:len(training)]

In [ ]:
#n= len(selected_with_SalePrice)
#colormap = plt.cm.viridis
#plt.figure(figsize=(n,n))
#plt.title('Pearson Correlation of Features', y=1.05, size=15)
#sns.heatmap(training.corr(),
#            linewidths=0.1,
#            vmax=1.0, 
#            square=True, 
#            cmap=colormap, 
#            linecolor='white', 
#            annot=True)

In [ ]:
new_total2 = dummify_category_cols(new_total,category_selected)

In [ ]:
new_total2.head(2)

In [ ]:
training = new_total2.iloc[:len(training)]
test = new_total2.iloc[len(training):]

In [ ]:
#여기서 selected 재설정

In [ ]:
training.head(2)

In [ ]:
training['is_train'] = np.random.uniform(0, 1, len(training)) <= .75

In [ ]:
training_set = training[training['is_train']==True]
test_set = training[training['is_train']==False]

In [ ]:
N = 3000
forest = RandomForestRegressor(n_estimators=N, n_jobs=2)

In [ ]:
feature = new_total2.columns.drop("SalePrice")
target = ["SalePrice"]

In [ ]:
forest.fit(training_set[feature], training_set[target])

In [ ]:
forest.predict(test_set[feature])

In [ ]:
forest.score(test_set[feature], test_set[target])

In [ ]:
y_pred=forest.predict(test_set[feature])

In [ ]:
y_pred = np.array(list(map(lambda x: math.exp(x)-1,y_pred)))

In [ ]:
y_true = test_set["SalePrice"]
y_ture= np.array(list(map(lambda x: math.exp(x)-1,y_true)))

In [ ]:
y_true = test_set["SalePrice"]
rmse = RMSE(y_true,y_pred)
print(rmse)

In [ ]:
importance = pd.DataFrame(forest.feature_importances_, index = training_set[feature].columns)

In [ ]:
(
    importance.assign(header = lambda x: x.index.str.split("_").str[0])
                  .groupby("header")[0]
                  .sum()
                  .iplot(kind="bar")
)

In [ ]:
#최적화

In [ ]:
# 최적화할 파라미터 N
best_score = 0
for N in range(4000,8500,500):
    tmp = RandomForestRegressor(n_estimators=N)
    tmp.fit(training_set[feature], training_set[target])
    score = tmp.score(test_set[feature],test_set[target])
    print("최적화 n_estimator 값 : {}".format(N))
    print("예측률 : {}".format(score))
    
    # 만약 기존의 모델보다 더 예측률이 높다면, 최신화합니다
    if score>best_score:
        best_score = score
        best_N = N
        best_rf_model = tmp
        
print("최적화 n_estimator 값 : {}".format(best_N))
print("예측률 : {}".format(best_score))

In [ ]:
#best_score = 0
#for N in range(13,21):
#    tmp = RandomForestRegressor(n_estimators=4500, max_depth = N)
#    tmp.fit(training_set[feature], training_set[target])
#    score = tmp.score(test_set[feature],test_set[target])
#    
#    # 만약 기존의 모델보다 더 예측률이 높다면, 최신화합니다
#    if score>best_score:
#        best_score = score
#        best_N = N
#        best_rf_model = tmp
        
#print("최적화 max_depth 값 : {}".format(best_N))
#print("예측률 : {}".format(best_score))

In [ ]:
#제출

In [ ]:
forest.fit(training[feature], training[target])

In [ ]:
prediction =forest.predict(test[feature])

In [ ]:
prediction = np.array(list(map(lambda x: math.exp(x)-1,prediction)))

In [ ]:
submission = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction})

In [ ]:
submission.to_csv("Submission37.csv")